# LCA with uniform uncertainty 

Steps for Performing LCA with Uniform Uncertainty:
1. Prepare the input files.
2. Import the product system.
3. Create uncertainty arrays for datapackages.
4. Create datapackages with uncertainty information.
5. Perform the Life Cycle Assessment (LCA).

1. Navigate to the project that contains the ecoinvent database (ecoinvent database is not necessary if you don't need to find characterization factors through Brightway).

In [1]:
import bw2data as bd

bd.projects.set_current("advlca25")
bd.databases

Databases dictionary with 4 object(s):
	ALIGNED-biob-prod-dummy
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential
	exldb

2. Import required libraries

In [2]:
import os
from bamboo_lca.background_importer import *
from bamboo_lca.foreground_importer import *
from bamboo_lca.datapackage_builder import *
from bamboo_lca.uncertainty_handler import *
from bamboo_lca.lca_wrapper import *
from bamboo_lca.uncertainty_importer import *

3. Define required constants

In [3]:
# BACKGROUND DATABASE FILE PATH
EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/S.txt")

# FOREGROUND DATABASE FILE PATH
FOREGROUND_FILE = os.path.join(os.getcwd(), "data/foreground_system.csv")

# CHARACTERIZATION FACTOR MAPPING FILE PATH
CF_MAPPING_FILE = os.path.join(os.getcwd(), "data/cf_mapping_file.csv")

# LCIA METHOD
METHOD = ('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global temperature change potential (GTP100)')

# CHOSEN BIOSPHERE EMISSIONS
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - fossil - air",
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

4. Import the background system. (Click [here](1.%20lca_with_background.ipynb) for more detailed explanation.)

In [4]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.build_tech_matrix(raw_tech)

bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.build_bio_matrix(bio_df, GHG)

cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG)

All characterization factors have been found.


5. Import the foregorund system. (Click [here](2.%20lca_with_foreground.ipynb) for more detailed explanation.)

In [5]:
# Find all activities in the product system.
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
fg_activities = get_fg_activities(FOREGROUND_FILE, ",", bg_activities)
activities = fg_activities + bg_activities

# Import the foreground system
fg_tech_df = pd.read_table(FOREGROUND_FILE, sep=',')
fg_dataframe = get_fg_dataframe(fg_tech_df, fg_activities)

fg_importer = ForegroundImporter()
fgbg, fgfg, bgfg, bifg = fg_importer.extend_matrix(fg_dataframe, GHG, fg_activities, bg_activities)
full_tech_matrix, full_bio_matrix = fg_importer.concatenate_matrix(tech_matrix, bio_matrix, fgbg, fgfg, bgfg, bifg)

6. Prepare datapackage matrix data

In [6]:
dp_builder = DatapackageBuilder()

(tech_data, tech_indices, tech_flip), (bio_data, bio_indices), (cf_data, cf_indices) = dp_builder.prepare_dp_matrix(full_tech_matrix, full_bio_matrix, cf_matrix)
datapackage_data = (tech_data, tech_indices, tech_flip), (bio_data, bio_indices), (cf_data, cf_indices)

7. Prepare datapackage uncertainty data

In [7]:
# Set uncertainty
uncertainty_type = 2
uncertainty_value = 1.106

# Create uncertainty arrays
uncertainty_handler = UncertaintyHandler()
tech_uncertainty_array = uncertainty_handler.add_uniform_uncertainty(uncertainty_type, uncertainty_value, True, tech_data, tech_flip)
bio_uncertainty_array = uncertainty_handler.add_uniform_uncertainty(uncertainty_type, uncertainty_value, True, bio_data)
uncertainty = [tech_uncertainty_array, bio_uncertainty_array, None]

8. Create the datapackage

In [8]:
dp = dp_builder.prepare_datapackage(datapackage_data, uncertainty)

9. Set functional unit

In [9]:
# Set "RoW-Services" as functional unit. (NOTE: By default, the region and industry name are joined with a dash.)
selected_activity = "RoW-Services"

# Find the index of the activity in activities.
index = activities.index(selected_activity)

# Define the functional unit
functional_unit = {index: 1}
print(f"The functional unit is {functional_unit}")

The functional unit is {71: 1}


10. Perform LCA

In [11]:
lca = bc.LCA(
            demand=functional_unit,
            data_objs=[dp],
            use_distributions=True,
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {activities[index]}, {lca.score}")

Brightway calculated lca score: RoW-Services, 8824.900772259492
